In [1]:
# %%html
# <script defer src="https://use.fontawesome.com/releases/v5.0.8/js/all.js" integrity="sha384-SlE991lGASHoBfWbelyBPLsUlwY1GwNDJo3jSJO04KZ33K2bwfV9YBauFfnzvynJ" crossorigin="anonymous"></script>
# <script disableClusteringAtZoom: 11 to L.markerClusterGroup({})></script>

<i>L'application "<b><u>Lieux de stages</u></b>" s'installe. Patientez..</i>

In [2]:
from ipywidgets import *
from IPython.display import IFrame, YouTubeVideo
from IPython.core.display import display
import subprocess, sys
import webbrowser
from ipywidgets import HTML
import base64
import csv ## https://docs.python.org/fr/3.6/library/csv.html 
import os ## Pour utiliser la methode ci-dessous Get Current Worked Directory pour lire la page Web crée dans le dossier courant.
Lieu = os.getcwd()  ## methode Get Current Worked Directory pour lire la page Web crée dans le dossier courant.
###
import os

import subprocess, sys

from time import gmtime, strftime
from datetime import datetime
from pytz import timezone

from reportlab.lib.enums import TA_JUSTIFY
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
#################################################
def Recup_Donnees(titre_csv):
    """ Cette fonction initiale et essentielle permet de construire
        une liste de tuples contenant toutes les informations du 
        tableau par lignes. """
    Donnees=[]  ## liste de tuples pour récupérer toutes les données du fichier csv
    Ligne_Donnees=[]  ## sous liste pour créer la liste des tuples de Donnees
    with open(titre_csv, "r", encoding="utf-8") as texte:  ## Ouverture du csv en mode utf 8 pour encodage des caractères français
        Les_lignes = csv.reader(texte, delimiter=';') ## lecture de chaque ligne du fichier csv
        for ligne in Les_lignes:  ## bouclage sur chaque ligne des lignes
            for element in ligne:  ## Récupération de chaque élément de chaque ligne
                Ligne_Donnees.append(element)  ## création d'1 tuple avec tous les éléments d'1 même ligne
            Ligne_Donnees=tuple(Ligne_Donnees)  ## assurance qu'il s'agit bien d'un tuple ..?
            Donnees.append(Ligne_Donnees)  ## Ajout du tuple créé à la liste de tuple Données
            Ligne_Donnees=[]  ## Réinitialisation d'1 futur 'tuple' vide pour boucler sur la ligne suivant du tableau
    longueur_Donnees=len(Donnees)  ## Détermination de la longueurs des Données
    return Donnees,longueur_Donnees
Donnees,longueur_Donnees=Recup_Donnees("Entreprises_Complet2.csv")
# print("longueur = ",longueur_Donnees)
Nb_entreprise=1

for ligne in range(0,longueur_Donnees):
    tupl=Donnees[ligne]
    li=list(tupl)
#     print(" ***** Ligne = ",ligne," ***********")
#     print()
#     print(li)
    result_Nom_entreprise=li[45] ##yes
    if result_Nom_entreprise != "":
        Nb_entreprise += 1
# print("Nb_entreprise = ",Nb_entreprise)

In [3]:
sel = []
sel_html = HTML()
sel_html.value = ""
def select_entreprises_fonc(change):
    global selection
    sel=change['new']
    sel=list(sel)
#     ty=type(sel)
#     print("type =",ty)
#     print("selection = ",sel)
    selection = sel
     ###############
    new_val = ""
    for az in range(0,len(sel)):
        new_val = new_val+ "<br>" + str(sel[az])
    sel_html.value = new_val
    ##############
    print("sel_html =",sel_html)
    return selection, sel , sel_html

In [4]:
def stop_video(_):
    out.layout.display = "none"
    Mode_w.value="---"
btn_exit_video = Button(description="Enlever la vidéo",tooltip="Cliquer pour enlever la vidéo",button_style='primary')
btn_exit_video.on_click(stop_video)
############
vid=YouTubeVideo( 'EKo7lR1lvSY')#'eWzY2nGfkXk')
# https://youtu.be/EKo7lR1lvSY
# Box_Vid=HBox([vid,btn_exit_video])
out = widgets.Output(layout={'border': '1px solid black'})
display(out)

with out:
    display(vid)
    display(btn_exit_video)
out.layout.display = "none"  #.layout.visibility = 'hidden'

#####
from IPython.display import IFrame

Output(layout=Layout(border='1px solid black'))

In [5]:
def stop_infos(_):
    out_infos.layout.display = "none"
    Mode_w.value="---"
btn_exit_infos = Button(description="Enlever les informations récoltées",tooltip="Cliquer pour enlever les informations récoltées",button_style='primary')
btn_exit_infos.on_click(stop_infos)
############
out_infos = widgets.Output(layout={'border': '1px solid black'})
display(out_infos)
with out_infos:
    display(sel_html)
    display(btn_exit_infos)
out_infos.layout.display = "none"  #.layout.visibility = 'hidden'
from IPython.display import IFrame

Output(layout=Layout(border='1px solid black'))

In [6]:
from ipyleaflet import (
    Map,
    Marker, MarkerCluster,LayersControl,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    Popup,
    GeoJSON,
    DrawControl,
    basemaps, FullScreenControl,
    SearchControl, AwesomeIcon, Icon, ScaleControl
)

from ipywidgets import HTML

In [7]:
List_Nom_Entrprise = []
for ligne in range(1,Nb_entreprise+3):
    tupl=Donnees[ligne]
    li=list(tupl)
    result_Nom=li[2]
    List_Nom_Entrprise.append(result_Nom)
List_Nom_Entrprise.sort()

select_entreprises = widgets.SelectMultiple(
    options=List_Nom_Entrprise ,
    #value=['Oranges'],
    rows=5,

    disabled=False,
    layout={'width': 'max-content'}
)
select_entreprises.observe(select_entreprises_fonc,names='value')
# display(select_entreprises)
#     description='Entreprises',

In [8]:
Selecti = VBox([widgets.Label(value='1) Choisissez, ci-dessous, vos entreprises (Ctrl pour plusieurs).'),select_entreprises])

In [9]:
def Create_pdf(_):
    global selection
    print("apres btn pdf, selection =",selection)
    infos = []
    Long_selection = len(selection)
    for select in range (Long_selection):
        for ligne in range(1,Nb_entreprise+3): # 19):
            tupl=Donnees[ligne]
            li=list(tupl)
            result=li[2] 
            if selection[select] == result:               
                infos.append(str(li[2])) 
                infos.append(str(li[0]))
                infos.append(str(li[1]))
                infos.append(str(li[40])) 
                infos.append(str(li[41])) 
                infos.append(str(li[42])) 
                infos.append("--> Tel : "+str(li[43])) 
                infos.append("--> Mail : "+str(li[51])) 
                infos.append("--> Lien : "+str(li[44])) 
                infos.append("**")
    Story = ""
    ###
    fmt = "%Y-%m-%d %H:%M" 
    now_utc = datetime.now(timezone('UTC'))
    formatted_time = now_utc.astimezone(timezone('Europe/Berlin'))    
    ###
    new_formatted_time=""
    long_string=len(str(formatted_time))
    b=str(formatted_time)
    for a in range(16):
        new_formatted_time=new_formatted_time+b[a]    
    ####
    Story = Story + "Informations sur les lieux de stages sélectionnés à la date de : "+ new_formatted_time
    Story = Story + "<br>----------------------------------------------------------------------------------------------"
    Story = Story + "<br>"
    for part in infos:
        ptext = '%s' % part.strip()
        if ptext == "**":
            Story = Story + ptext + "<br><br>"
        else:
            Story= Story + ptext + "<br>" 
    sel_html.value = Story

In [10]:
#######################################
btn_pdf = Button(icon="file",button_style='danger') # cogs  # angle-left # file-pdf-o # spinner
btn_pdf.layout.width="30px"
btn_pdf.on_click( Create_pdf)
# display(btn_pdf)
#################

In [11]:
def Vid_fonc(_):
    out.layout.display = "block"
#####################
Vid_w = widgets.ToggleButton(
    value=False,
    description=' Démo',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Cliquer pour voir la vidéo",
    icon='video' # (FontAwesome names without the `fa-` prefix)
)
Vid_w.observe(Vid_fonc,names='value')
#####################
# out.layout.display = "block"

In [12]:
def infos_fonc(_):
    out_infos.layout.display = "block"
#####################
# yep = widgets.ToggleButton(
#     value=False,
#     description=' Infos',
#     disabled=False,
#     button_style='info', # 'success', 'info', 'warning', 'danger' or ''
#     tooltip="Cliquer pour voir les infos",
# #     icon='video' # (FontAwesome names without the `fa-` prefix)
# )
# yep.observe(infos_fonc,names='value')
#####################
yep = Button(icon="eye",button_style='danger',tooltip="Cliquer pour voir les infos") # cogs  # angle-left # file-pdf-o # spinner
yep.layout.width="30px"
yep.on_click(infos_fonc)
# out.layout.display = "block"

TypeError: on_click() got an unexpected keyword argument 'names'

In [ ]:
for a in range(1,Nb_entreprise+2): # 19):
    tuplee=Donnees[a]
    lib=list(tuplee)
    if lib[0]=="Lycée":
        ligne=a
    
# ligne=1
tupl=Donnees[ligne]
li=list(tupl)
Nom_lycee=li[2]
#################################
Nb="L'établissement '"+str(Nom_lycee)+"' propose "+str(Nb_entreprise-2)+" adresses de lieux de stages :"
Nb2 = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p><i>'+ Nb +' </i></p>')
Nb3=VBox([Nb2,Vid_w])
# Vid_w
Nb3.layout.max_width='200px'
Nb3.layout.border = '1px solid black'

In [ ]:
#################################
Tex2="Créez votre sélection d'informations des lieux de stages :"
Texto2 = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p><u><b>'+ Tex2 +' </b></u></p>')
# Tex1a = "1) Choisissez vos entreprises (Ctrl pour plusieurs)."
# Texto1a = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Tex1a +' </p>')

# Tex1b = "2) Cliquez sur le bouton 'Creez le PDF'."
# Texto1b = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Tex1b +' </p>')

# Tex1c = "3) Cliquez sur le bouton 'Ouvrir de PDF'."
# Texto1c = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Tex1c +' </p>')


Tex3="2) Créer visuel --> "
Texto3 = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Tex3 +' </p>')
Btn_CrPDF = HBox([Texto3,btn_pdf])

Tex4="3) Présenter visuel --> "
Texto4 = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ Tex4 +' </p>')
Btn_OuvrPDF = HBox([Texto4,yep])

Btn = VBox([Btn_CrPDF,Btn_OuvrPDF])
Boite = HBox([Selecti, Btn])

Texte_w = VBox([Texto2,Boite]) # Texto1a,Texto1b,Texto1c, 

Texte_w.layout.max_width='640px'
Texte_w.layout.border = '1px solid black'
#################################
# Texte_w
####################################
# TexDrop1="Choisissez les domaines d'entreprises"
# TexDrop1a = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p><u><b>'+ TexDrop1 +' </b></u></p>')
# TexDrop2="visibles sur la carte :"
# TexDrop2a = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p><u><b>'+ TexDrop2 +' </b></u></p>')
# dropi=VBox([TexDrop1a,TexDrop2a,drop])
# dropi.layout.max_width='340px'
# dropi.layout.border = '1px solid black'

In [ ]:
file = open("Lib2.png", "rb")
image = file.read()
im_lycee = widgets.Image(
    value=image,
    format='png',
    width=200,
    height=200,
)


In [ ]:
file = open("titre2.png", "rb")
image = file.read()
im_titre = widgets.Image(
    value=image,
    format='png',
    width=400,
    height=50,
)
im_titre
Box_im_titre=Box([im_titre])
Box_im_titre.layout.border = '1px solid black'
Box_im_titre

In [ ]:
Menus = HBox([im_lycee,Nb3,Texte_w], layout=Layout(max_width='100%',display='inline-flex',flex_flow='row wrap'))
display(Menus)

In [ ]:
from ipyleaflet import (
    Map, basemaps, basemap_to_tiles,
    Circle, Marker, Rectangle, LayerGroup, LayersControl,
    AwesomeIcon, Popup, ScaleControl, FullScreenControl,
    SearchControl
)
from ipywidgets import HTML
###################################
names = []
##################################
######## Récup des coordonnées du lycée du fichier excel sinon Libergier ######
faire = 1
for ligne in range(1,Nb_entreprise+3):
    tupl=Donnees[ligne]
    li=list(tupl)
    resulto=li[0] ##yes
    if resulto =="Lycée":
        result=li[45]
        if result !="":
            result=result.split(",")
            latitude=result[0]
            longitude=result[1]
            cent = [latitude, longitude]

            faire = 0
if faire==1:
    print("Manque les coordonnées à votre lycée.")
    cent = [49.25112, 4.0271] #drop.value#
###############################################
zoom = 20
m = Map(center=cent, zoom=zoom)
m.add_control(FullScreenControl())
marker = Marker(icon=AwesomeIcon(name="check", marker_color='green', icon_color='darkgreen'))

m.add_control(SearchControl(
  position="topleft",
  url='https://nominatim.openstreetmap.org/search?format=json&q={s}',
  zoom=15,
  marker=marker
))

m.interact(zoom=(5,20,1))
###############################################################
""" Création de la liste (names) des différents domaines d'entreprise"""
for ligne in range(1,Nb_entreprise+3):
    faire = 1
    tupl=Donnees[ligne]
    li=list(tupl)
    result=li[45] ##yes
    if result !="":
        long_names=len(names)
        for i in range(long_names):
            if names[i]==li[0]:
                faire=0
        if faire==1:
            names.append(li[0])
###########################################################
## Début Modif 2022 N°1 ##
Liste_Entreprises_Annees_retenues = [  ["","","","","","" ] for i in range(Nb_entreprise+3)  ] ## limitte années
""" Ajout à la liste (names) des 5 dernières années Colonne D du tableau """
now_utc = datetime.now(timezone('UTC'))
formatted_time = now_utc.astimezone(timezone('Europe/Berlin'))    
Annee_en_cours=""
long_string=len(str(formatted_time))
b=str(formatted_time)
for a in range(4):
    Annee_en_cours=Annee_en_cours+b[a]
for ligne in range(1,Nb_entreprise+3):
    faire = 1
    tupl=Donnees[ligne]
    li=list(tupl)
    result=li[3] ## colonne des années
    ## Début module repere Annees ##
    Nb_digit_annee = 0
    Annee_retenue = ""
    Liste_Annees_retenues = []
    Chiffre_ok = False
    List_Numeros = ['0','1','2','3','4','5','6','7','8','9']
    Longueur_Chain = len(result)
    for position in range(Longueur_Chain):
        Chiffre_ok = False
        Caractere = result[position]
        for j in range(len(List_Numeros)):
            Numero_dans_List_Numeros = List_Numeros[j]
            if Caractere == Numero_dans_List_Numeros:
                Chiffre_ok = True
                Nb_digit_annee = Nb_digit_annee +1
                if Nb_digit_annee == 4:
                    Nb_digit_annee = 0
                    Annee_retenue = Annee_retenue + Caractere
                    if Annee_retenue in Liste_Annees_retenues:
                        rien = 1
                    else:
                        Liste_Annees_retenues.append(Annee_retenue)
                    Annee_retenue = ""
                    Chiffre_ok = False
                else:
                    Annee_retenue = Annee_retenue + Caractere
        if Chiffre_ok == False:
            Nb_digit_annee = 0
            Annee_retenue = ""
    if Liste_Annees_retenues == []:
        Liste_Annees_retenues = [""]
        Liste_Annees_retenues.append(Annee_retenue)
    Liste_Annees_retenues = Liste_Annees_retenues[:6] ## limitte années
    for Index_annee in range(len(Liste_Annees_retenues)):
        annee = Liste_Annees_retenues[Index_annee]
        if annee != "":
            if int(annee) > int(Annee_en_cours) or int(annee) < (int(Annee_en_cours)-4): ## limitte controle
                rien = 1
                Liste_Entreprises_Annees_retenues[ligne][Index_annee]='0'
            else:
                Liste_Entreprises_Annees_retenues[ligne][Index_annee] = Liste_Annees_retenues[0+Index_annee]  
                if result !="":
                    long_names=len(names)
                    for i in range(long_names):
                        if names[i]==Liste_Annees_retenues[Index_annee]:
                            faire=0
                    if faire==1:
                        names.append(Liste_Annees_retenues[Index_annee])
## Fin Modif 2022 N°1 ##
"""Création des noms de liste de domaine nommées markers_domain"""
noms_domaine = []
long_names=len(names)
for name in range(long_names):
    nom="markers_"+names[name]
    noms_domaine.append(nom)
dico_noms_domaine = {n: [] for n in noms_domaine}
#############################################################
"""Création des noms de liste de couches nommées couches_domain"""
couches_domain = []
long_names=len(names)
for name in range(long_names):
    nom="couches_"+names[name]
    couches_domain.append(nom)
list_couches_domain = [n for n in couches_domain]
#############################################################
for ligne in range(1,Nb_entreprise+3): 
    tupl=Donnees[ligne]
    li=list(tupl)
    result=li[45] 
    if result !="":
        result=result.split(",")
        latitude=result[0]
        longitude=result[1]
        Coord = [latitude, longitude]

        infos="<u>Entreprise :</u> <b>"+str(li[2])+"</b> : <br />"+str(li[0])+" : "+str(li[1])+". <br />"+str(li[3])+". <br />"+str(li[40])+", "+str(li[41])+" "+str(li[42])+"<br/> --> Tel : "+str(li[43])+"<br/> --> Mail : "+str(li[51])+"<br/> --> <a target='_blank' rel='noopener noreferrer' href='"+str(li[44])+"'>Lien </a>"
        ## icon de base
        icon = AwesomeIcon(name='industry', marker_color='lightblue', icon_color='black', icon_size=[40, 40])

        if li[0] == "Lycée":
            icon = AwesomeIcon(name='fa-graduation-cap', marker_color='lightred', icon_color='black', icon_size=[40, 40])
        if li[0] == "Chimie":
            icon = AwesomeIcon(name='flask', marker_color='beige', icon_color='black' , icon_size=[40, 40])
        if li[0] == "Médical":
            icon = AwesomeIcon(name='heartbeat', marker_color='red', icon_color='black' , icon_size=[40, 40])
        if li[0] == "BCM":
            icon = AwesomeIcon(name='cut', marker_color='lightgray', icon_color='black' , icon_size=[40, 40])
        if li[0] == "Génétique":
            icon = AwesomeIcon(name='dna', marker_color='lightgray', icon_color='black' , icon_size=[40, 40])
        if li[0] == "Fermentations":
            icon = AwesomeIcon(name='hourglass-start', marker_color='lightgray', icon_color='black' , icon_size=[40, 40])
        if li[0] == "Microbiologie":
            icon = AwesomeIcon(name='link', marker_color='lightgray', icon_color='black' , icon_size=[40, 40])
        if li[0] == "Environnement":
            icon = AwesomeIcon(name='leaf', marker_color='lightgreen', icon_color='black' , icon_size=[40, 40])
        if li[0] == "Pharmaceutique":
            icon = AwesomeIcon(name='pills', marker_color='lightblue', icon_color='black' , icon_size=[40, 40])
        if li[0] == "Cosmétique":
            icon = AwesomeIcon(name='eye', marker_color='pink', icon_color='black' , icon_size=[40, 40])
        if li[0] == "Alimentaire":
            icon = AwesomeIcon(name='utensils', marker_color='orange', icon_color='black', icon_size=[40, 40])

##### Suite Modif 2022 N°2 ## A placer après les icônes
        New_List = ['*']
        for annee in Liste_Entreprises_Annees_retenues[ligne]:
            New_List.append(annee)
        Liste_Entreprises_Annees_retenues[ligne] = New_List
        for annee in Liste_Entreprises_Annees_retenues[ligne]:
            if annee == "*":
                mark = Marker(location=Coord, draggable=False, icon=icon, name=li[0]) 
            else:
                if annee == "":
                    break
                else:
                    new_name = li[0] + "_" + annee
                    mark = Marker(location=Coord, draggable=False, icon=icon, name=new_name)
        #########
            """Remplis les listes 'markers_domaine des différents points de marqueurs'"""
            for name in range(long_names):
                new_name = annee
                if li[0] == names[name] or new_name == names[name]:
                    listo = dico_noms_domaine[noms_domaine[name]]
                    listo.append(mark) 
                ##########################################
            message2 = HTML()
            message2.value = infos
            popup = Popup(
                location=Coord,
                close_button=False,
                auto_close=False,
                close_on_escape_key=False
            ) 
            mark.popup = message2
    #####################################################""
"""Remplis les listes 'markers_domaine des différents points de marqueurs'"""
for nume in range(long_names):
    name = (long_names-1)-nume
    listo = dico_noms_domaine[noms_domaine[name]]
    list_couches_domain[name] = LayerGroup(name =str(names[name]), layers=listo)
    m.add_layer(list_couches_domain[name])
m.add_control(ScaleControl(position='bottomleft'))
control = LayersControl(position='topleft')
m.add_control(control)
sel_html
m
###### Fin Modif 2022 N°2 ## A placer après les icônes